<a href="https://colab.research.google.com/github/papoularubra/clienton/blob/main/ClientOn_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip -q install google-genai

In [12]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [13]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [21]:
!pip install -U googlemaps

In [41]:
import googlemaps

# Substitua 'SUA_CHAVE_DE_API' pela sua chave de API real
api_key = userdata.get('GOOGLE_MAPS_API_KEY')
gmaps = googlemaps.Client(key=api_key)

# Agora você pode usar o objeto 'gmaps' para interagir com a API
# Exemplo de uma chamada à API de geocodificação:
try:
    geocode_result = gmaps.geocode('Avenida Paulista, São Paulo')
    print(geocode_result)
except Exception as e:
    print(f"Ocorreu um erro ao chamar a API: {e}")

[{'address_components': [{'long_name': 'Avenida Paulista', 'short_name': 'Av. Paulista', 'types': ['route']}, {'long_name': 'São Paulo', 'short_name': 'São Paulo', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'São Paulo', 'short_name': 'SP', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Brazil', 'short_name': 'BR', 'types': ['country', 'political']}], 'formatted_address': 'Av. Paulista, São Paulo - SP, Brazil', 'geometry': {'bounds': {'northeast': {'lat': -23.5525567, 'lng': -46.64398449999999}, 'southwest': {'lat': -23.5721549, 'lng': -46.6657058}}, 'location': {'lat': -23.5613545, 'lng': -46.6564943}, 'location_type': 'GEOMETRIC_CENTER', 'viewport': {'northeast': {'lat': -23.5525567, 'lng': -46.64398449999999}, 'southwest': {'lat': -23.5721549, 'lng': -46.6657058}}}, 'place_id': 'ChIJFaMK2shZzpQRanfJ8jGUn9U', 'types': ['route']}]


In [14]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from googlemaps import places
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [15]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [16]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [39]:
##########################################
# -- Agente 1: Buscador de Informações -- #
##########################################

def agente_buscador(topico, data_de_hoje):

# Busca no Google Places
    places_result = gmaps.find_place(input=topico, input_type='textquery', fields=['name', 'formatted_address', 'rating', 'user_ratings_total', 'photos'])


    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em due diligence de empresas.
        Sua tarefa é realizar uma investigação abrangente sobre a reputação e histórico da empresa indicada em topico.
        Para isso, utilize a ferramenta de busca do Google (google_search) para:
        - Identificar informações básicas da empresa:
        CNPJ, data de fundação/tempo de existência, endereço, nicho de mercado principal e outras informações cadastrais relevantes.
        - Consultar a reputação online:
        Buscar e analisar avaliações e reclamações sobre a empresa em topico no site Reclame Aqui (reclameaqui.com.br).
        Explorar outras plataformas de avaliação de consumidores e reputação de empresas (se disponíveis e relevantes para o contexto brasileiro).
        Investigar comentários em redes sociais da empresa ou sobre a empresa em topico.
        - Buscar notícias sobre a empresa em topico, fotos e avaliações no API Places do Google.
        - Verificar pendências legais e regulatórias:
        Pesquisar por processos judiciais e menções da empresa em topico no Jusbrasil e sites jurídicos oficiais.
        Verificar a existência de reclamações ou processos administrativos contra a empresa em topico
        no Procon e outros órgãos reguladores relevantes para o nicho da empresa.
        Formato: Você vai realizar toda essa pesquisa, mas só vai mostrar as informações básicas e fundamentais neste passo, em formato tabulado.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Empresa: {topico}\nData de hoje: {data_de_hoje}\nPlaces Result: {places_result}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [30]:
################################################
# --- Agente 2: Avaliador de reputação --- #
################################################

def agente_avaliador(topico, lancamentos_buscados):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash", #não foi possível usar um modelo mais avançado, mas seria o ideal

        # Inserir as instruções do Agente Avaliador #################################################
        instruction="""
        Você é um orientador de consumidores especializado em due diligence de empresas.
        Com base em
        1 - nas informações levantadas pelo agente buscador, e/ou
        2 - na análise dos requisitos de segurança do link indicado em tópico, se for um link
        (Caso o usuário peça informações a partir de um link, realize as ações anteriores cabíveis e também:
        - Identifique se o link é seguro.
        - Identifique se o link é de propriedade da empresa sugerida.
        - Verifique os selos de segurança do site, a política de privacidade, certificado SSL e outros critérios de segurança
        que ajudem a evitar que o usuário caia em um golpe.),
        você deve:
        - Usar a ferramenta de busca do Google (google_search) para indicar se a empresa é confiável ou não e
        elaborar uma justificativa para a afirmação, de maneira sucinta, linguagem clara e acessível.
        - Se possível, categorize os tipos de reclamações ou avaliações
        (por exemplo, problemas com produto, atendimento, entrega) para fornecer uma visão mais detalhada da reputação.
        - Mencione a data dos principais comentários e categorize como "Elogio", "Reclamação" ou "Neutro".
        Destaque a informação desse agente em relação aos outros, pois é a mais importante.
        Destaque a conclusão sobre a empresa ou site ser confiável ou não colocando as expressões "É CONFIÁVEL" ou "NÃO É CONFIÁVEL"
        em negrito e caixa alta, esta informação deve estar na primeira linha do texto.
        Formato: As informações devem ser sucintas, com linguagem clara e acessível, apresentadas de  modo tabulado
        com os dados positivos em verde e os negativos em vermelho.
        """,
        description="Agente que avalia a confiabilidade da empresa",
        tools=[google_search]
    )

    entrada_do_agente_avaliador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    avaliacao_da_empresa = call_agent(avaliador, entrada_do_agente_avaliador)
    return avaliacao_da_empresa

In [19]:
######################################
# --- Agente 3: Consultor --- #
######################################
def agente_consultor(topico, avaliacao_da_empresa):
    consultor = Agent(
        name="agente_consultor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um consultor de consumidores, com foco em oferecer respostas claras e concisas para ajudá-los
            no consumo consciente e seguro diante da escolha de uma empresa.
            Com base nas informações dos lacamentos_buscados e avaliacao_da_empresa, você deve:
            - Considerar o tom geral das avaliações e reclamações encontradas pelo agente_buscador e agente_avaliador.
            - Indicar ao cliente se deve ou não negociar com a empresa ou comprar um produto dela.
            - Indicar para qual público e finalidade a empresa é indicada.
            - Se cabível, indicar aspectos de responsabilidade social, compliance, sustentabilidade e
            outras práticas que forem um diferencial da empresa.
            - Se a empresa for CONFIÁVEL: mencionar pontos positivos mais relevantes com base na recorrência do tema nas avaliações.
            - Se cabível, indicar concorrente para comparação, apenas se tiver qualidades compatíveis ou superiores às da empresa pesquisada.
            - Compare a reputação da empresa consultada com a média do setor, se houver dados disponíveis.
            - Considere que empresas sem informações cadastrais online, como CNPJ, não são confiáveis.
            Formato: Essa conclusão deve ser sucinta, com linguagem clara e acessível, ser tabulada, os dados positivos em verde e os negativos em vermelho.
            """,
        description="Agente consultor de negocios com foco no cliente"
    )
    entrada_do_agente_consultor = f"Tópico: {topico}\nPlano de post: {avaliacao_da_empresa}"
    # Executa o agente
    decisao_consumidor = call_agent(consultor, entrada_do_agente_consultor)
    return decisao_consumidor

In [44]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🧐 Iniciando o Sistema de Avaliação de Confiança de Empresas 🧐")

# --- Obter o Tópico do Usuário ---
topico = input("Digite o link do site ou o nome da empresa sobre a qual você deseja informações: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos saber mais sobre a empresa {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Informações sobre a empresa ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    avaliacao_da_empresa = agente_avaliador(topico, lancamentos_buscados)
    print("\n---❓Análise de confiança ---\n")
    display(to_markdown(avaliacao_da_empresa))
    print("--------------------------------------------------------------")

    decisao_consumidor = agente_consultor(topico, avaliacao_da_empresa)
    print("\n--- 👨‍⚖️ Sugestão de decisão para o consumidor ---\n")
    display(to_markdown(decisao_consumidor))
    print("--------------------------------------------------------------")



🧐 Iniciando o Sistema de Avaliação de Confiança de Empresas 🧐
Digite o link do site ou o nome da empresa sobre a qual você deseja informações: https://www.alura.com.br
Maravilha! Vamos saber mais sobre a empresa https://www.alura.com.br

--- 📝 Informações sobre a empresa ---



> Aqui está uma pesquisa abrangente sobre a reputação e o histórico da Alura:
> 
>  
> Com base nas minhas buscas, aqui está um resumo das informações sobre a Alura:
> 
>  | Informação | Detalhes |
>  |---|---|
>  | **Nome** | Alura (Caelum Ensino e Inovação) |
>  | **CNPJ** | Não encontrei o CNPJ diretamente, mas a Alura faz parte de um grupo maior que inclui a FIAP e a PM3. |
>  | **Data de Fundação** | A Alura foi lançada em 2013. A Caelum, empresa que deu origem à Alura, foi criada em 2004. A Alura Para Empresas foi criada em 2015. |
>  | **Endereço** |  Não encontrei o endereço exato da sede. |
>  | **Nicho de Mercado** |  Educação em tecnologia (cursos online). A Alura oferece cursos em diversas áreas como programação, front-end, data science, inteligência artificial, DevOps, UX & Design, Mobile e Inovação & Gestão. |
>  | **Reputação Online** |  Existem opiniões mistas sobre a Alura. Alguns usuários relatam cursos desatualizados e conteúdo fraco, enquanto outros elogiam a variedade de cursos e a possibilidade de aprender sobre diversas áreas da tecnologia. |
>  | **Reclame Aqui** | Não foi possível obter informações específicas sobre a Alura no Reclame Aqui durante a busca. |
>  | **Processos Judiciais** |  Não foram encontradas informações relevantes sobre processos judiciais envolvendo a Alura. |
>  | **Procon** | Não foram encontradas reclamações ou processos administrativos contra a Alura no Procon durante a busca. |
> 
>  **Observações:**
> 
>  *  A pesquisa encontrou algumas discussões online (Reddit) onde usuários expressam opiniões diversas sobre a Alura. Alguns consideram que o conteúdo pode ser desatualizado ou não tão aprofundado quanto em outras plataformas (como a Udemy).
>  *  A Alura oferece mais de 1.300 cursos gratuitos através de uma parceria com o Sebrae.
>  *  A Alura faz parte de um ecossistema de ensino que inclui a FIAP e a PM3.
>  *  É importante notar que algumas informações podem estar desatualizadas, já que alguns resultados da pesquisa são de 2022 e 2023.
> 


--------------------------------------------------------------

---❓Análise de confiança ---



> **É CONFIÁVEL**
> 
> Com base nas informações coletadas, a Alura aparenta ser uma empresa confiável, embora com algumas ressalvas em relação à qualidade do conteúdo de alguns cursos.
> 
> Aqui está uma análise mais detalhada:
> 
> *   **Informações Gerais:**
> 
>     *   Nome: Alura (Caelum Ensino e Inovação)
>     *   Data de Fundação: 2013 (Alura), 2004 (Caelum)
>     *   Nicho de Mercado: Educação em tecnologia (cursos online)
> *   **Reputação Online:**
> 
>     *   Mista. Alguns usuários relatam cursos desatualizados e conteúdo fraco.
>     *   Outros elogiam a variedade de cursos e a possibilidade de aprender sobre diversas áreas da tecnologia.
> *   **Reclame Aqui:** Informações não encontradas diretamente durante a busca.
> *   **Processos Judiciais/Procon:** Não foram encontradas informações relevantes.
> *   **Outras Observações:**
>     *   Possui mais de 1.300 cursos gratuitos em parceria com o Sebrae.
>     *   Faz parte de um ecossistema de ensino que inclui FIAP e PM3.
> 
> Para complementar a avaliação, realizei uma busca adicional para verificar a situação atual da Alura no Reclame Aqui e em outras plataformas de avaliação.
> 
> 
> Com base nas informações adicionais encontradas:
> 
> *   **Reclame Aqui:** A Alura possui uma reputação "ÓTIMA" no Reclame Aqui, com nota 7.6/10 (conforme consulta em maio de 2025).
> *   **Avaliações:** As avaliações online são mistas, com alguns usuários elogiando a plataforma e outros criticando a qualidade e atualização de alguns cursos.
> *   **Segurança:** O site da Alura possui certificado SSL válido, o que garante uma conexão segura.
> 
> Considerando todos os pontos, a Alura **É CONFIÁVEL**, mas é importante estar ciente das possíveis desatualizações em alguns cursos. Recomenda-se pesquisar e verificar a ementa e as avaliações de cada curso antes de se inscrever.
> 


--------------------------------------------------------------

--- 👨‍⚖️ Sugestão de decisão para o consumidor ---



> Com base na sua análise, aqui está uma conclusão para auxiliar o cliente na decisão sobre a Alura:
> 
> | Aspecto        | Avaliação                                                                                                                                                                                                                                                                                                                                                                                                                      |
> | :------------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
> | **Confiabilidade** | ✅ **CONFIÁVEL:** A Alura (Caelum Ensino e Inovação) é uma empresa estabelecida desde 2004, com certificado SSL válido.                                                                                                                                                                                                                                                                                                                   |
> | **Reputação**    | 🟢 **POSITIVO:** Possui reputação "ÓTIMA" no Reclame Aqui (7.6/10). <br> 🔴 **NEGATIVO:** Avaliações mistas quanto à atualização e qualidade de alguns cursos.                                                                                                                                                                                                                                                                  |
> | **Público/Finalidade** | Ideal para quem busca uma ampla variedade de cursos em tecnologia e a possibilidade de explorar diversas áreas. Indicada tanto para iniciantes quanto para profissionais que desejam se manter atualizados.                                                                                                                                                                                                                         |
> | **Diferenciais**   | ✅ Possui mais de 1.300 cursos gratuitos em parceria com o Sebrae, ampliando o acesso à educação. <br> ✅ Integra um ecossistema de ensino que inclui FIAP e PM3, o que pode oferecer mais oportunidades de aprendizado e networking.                                                                                                                                                                                                   |
> | **Recomendação**   | 👍 Recomenda-se pesquisar e verificar a ementa e as avaliações de cada curso antes de se inscrever, devido a relatos de desatualização em alguns casos.                                                                                                                                                                                                                                                                              |
> | **Alternativas**   | Para uma comparação mais completa, o cliente pode considerar plataformas como a Udemy ou Coursera, que também oferecem uma vasta gama de cursos em tecnologia e áreas correlatas.                                                                                                                                                                                                                                                      |
> | **Resumo**       | A Alura é uma plataforma confiável e bem avaliada, mas é **crucial** que o consumidor verifique a avaliação e grade do curso antes de adquirir o serviço, visando garantir que o conteúdo esteja alinhado com suas necessidades e expectativas. A reputação da empresa no Reclame Aqui é um indicador positivo, mas a diligência individual é fundamental.                                                                              |


--------------------------------------------------------------
